In [ ]:

# !pip install pandas plotly pillow scikit-learn matplotlib

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
import plotly.graph_objects as go
import plotly.io as pio
from PIL import Image
import base64
from io import BytesIO


In [ ]:

file_path = "base de dados pará.xlsx"
excel_file = pd.ExcelFile(file_path)
df = excel_file.parse('Planilha1')


In [ ]:

df_long = df.melt(id_vars=["transition_name"],
                 value_vars=[year for year in range(1986, 2022)],
                 var_name="Ano", value_name="Area_ha")
df_agg = df_long.groupby(["transition_name", "Ano"]).sum().reset_index()
df_agg_filtered = df_agg[df_agg['Ano'].between(1989, 2021)]
df_agg_filtered = df_agg_filtered[~df_agg_filtered['transition_name'].isin(["Ignorado", "Outras transições"])]


In [ ]:

logo_path = "Logo ZettaLab.png"
logo_image = Image.open(logo_path)
logo_image = logo_image.resize((200, 50))
buffered_logo = BytesIO()
logo_image.save(buffered_logo, format="PNG")
logo_base64 = base64.b64encode(buffered_logo.getvalue()).decode()


In [ ]:

fig = go.Figure()
for transition in df_agg_filtered['transition_name'].unique():
    df_filtered = df_agg_filtered[df_agg_filtered['transition_name'] == transition]
    fig.add_trace(go.Scatter(x=df_filtered['Ano'], y=df_filtered['Area_ha'],
                             mode='lines+markers', name=transition, visible=True))


In [ ]:

def treinar_modelo(df, categoria):
    df_cat = df[df['transition_name'] == categoria]
    X = df_cat['Ano'].values.reshape(-1, 1)
    y = df_cat['Area_ha'].values
    modelo = LinearRegression()
    modelo.fit(X, y)
    return modelo

modelo_antr = treinar_modelo(df_agg_filtered, 'Antrópico')
modelo_veg_prim = treinar_modelo(df_agg_filtered, 'Veg. Primária')

anos_proj = np.arange(2022, 2101).reshape(-1, 1)
projecao_antr = modelo_antr.predict(anos_proj)
projecao_veg_prim = modelo_veg_prim.predict(anos_proj)

diferencas = projecao_antr - projecao_veg_prim
ano_cruzamento_idx = np.where(diferencas > 0)[0]
ano_cruzamento = anos_proj[ano_cruzamento_idx[0]][0] if len(ano_cruzamento_idx) > 0 else None


In [ ]:

# Botão de projeção (mostra apenas as linhas da projeção)
button_projecao = dict(
    label="🔍 Ver Projeção 2100 (Clique Aqui)",
    method="update",
    args=[
        {"visible": [False] * len(df_agg_filtered['transition_name'].unique()) + [True, True]},
        {"title": "Projeção: Quando as Áreas Antrópicas Podem Ultrapassar a Vegetação Primária no Pará (1989-2100)"}
    ]
)

# Botão de volta ao dashboard principal
button_voltar = dict(
    label="🔙 Voltar ao Dashboard",
    method="update",
    args=[
        {"visible": [True] * len(df_agg_filtered['transition_name'].unique()) + [False, False]},
        {"title": "DESMATAMENTO E VEGETAÇÃO SECUNDÁRIA NO PARÁ (1989-2021)"}
    ]
)


In [ ]:

fig.add_trace(go.Scatter(x=anos_proj.flatten(), y=projecao_antr, mode='lines', name='Antrópico (Projeção)', visible=False))
fig.add_trace(go.Scatter(x=anos_proj.flatten(), y=projecao_veg_prim, mode='lines', name='Vegetação Primária (Projeção)', visible=False))


In [ ]:

fig.update_layout(
    paper_bgcolor="#0d2950",
    plot_bgcolor="#0d2950",
    font=dict(color="white"),
    title=dict(text="DESMATAMENTO E VEGETAÇÃO SECUNDÁRIA NO PARÁ (1989-2021)", x=0.5, xanchor='center', font=dict(size=20, family="Arial", color="white")),
    images=[dict(source="data:image/png;base64," + logo_base64, xref="paper", yref="paper",
                 x=0, y=1.15, sizex=0.15, sizey=0.15, xanchor="left", yanchor="top", layer="above")],
    updatemenus=[dict(type="buttons", direction="down", buttons=[button_projecao, button_voltar], x=1.15, y=1.2)]
)

# Exportar
pio.write_html(fig, "dashboard_desmatamento_com_projecao_integrada.html")
